<a href="https://colab.research.google.com/github/sidneyarcidiacono/fastai-practical-deep-learning/blob/main/notebooks/snake_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OSError: ignored

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
import os
key = os.environ.get('AZURE_SEARCH_KEY', 'bd220979600745d3b0f6aa712b11d062')

In [ ]:
results = search_images_bing(key, 'python')
ims = results.attrgot('content_url')
len(ims)

In [ ]:
#hide
ims = ['https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fthumb%2Fb%2Fb4%2FPython_reticulatus_%25D1%2581%25D0%25B5%25D1%2582%25D1%2587%25D0%25B0%25D1%2582%25D1%258B%25D0%25B9_%25D0%25BF%25D0%25B8%25D1%2582%25D0%25BE%25D0%25BD-2.jpg%2F1200px-Python_reticulatus_%25D1%2581%25D0%25B5%25D1%2582%25D1%2587%25D0%25B0%25D1%2582%25D1%258B%25D0%25B9_%25D0%25BF%25D0%25B8%25D1%2582%25D0%25BE%25D0%25BD-2.jpg&f=1&nofb=1']

In [ ]:
dest = 'drive/MyDrive/snake-images/python.jpg'
download_url(ims[0], dest)

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
snake_types = 'python','viper','cobra'
path = Path('snakes')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in snake_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} snake')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
snakes = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = snakes.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=6, nrows=1)

In [ ]:
snakes = snakes.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = snakes.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
snakes = snakes.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = snakes.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('drive/MyDrive/snake-images/python.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])